<a href="https://colab.research.google.com/github/Keerthana123-coder/Automated-Waste-Segregation-with-Computer-Vision/blob/main/AI_Powered_Waste_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install --quiet  scikit-learn opencv-python-headless ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.5 MB/s eta 0:00:00


In [11]:
# ===========================
# STEP 1: Setup & Dependencies
# ===========================

!pip install kaggle mediapipe opencv-python matplotlib tensorflow==2.19.0

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import mediapipe as mp


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
!unzip -q waste-classification-data.zip -d ./dataset
